## More Data Processing with Pandas

### Pivot Tables

A pivot table is a way of summarising data in a DataFrame for a particular purpose. It makes heavy use of the aggregation function. A pivot table is itself a DataFrame, where the rows represent one variable that you're interested in, the columns another, and the cells contain some aggregate value.

A pivot table also tends to include marginal values, which are the sumsof each column and row. This allows you to be able to see the relationship between two variables at just a glance.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Let's work with the Times Higher Education World University Ranking dataset
df = pd.read_csv('../resources/week-3/datasets/cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


We can create a new column called `Rank_Level`, where institutions with world ranking 1-100 are categorised as first tier and those with world ranking 101-200 are second tier, ranking 201-300 are third tier, and after 301 is other top universities. Remember how we can do this?

Steps:

1. Create a function to create the categories according to the institution's ranking

2. Apply this function to a single column of data to create a new Series

In [3]:
# Create categories
def create_category(ranking):

    if ranking >= 1 and ranking <= 100:
        return '1st Tier Top University'
    elif ranking >= 101 and ranking <=200:
        return '2nd Tier Top University'
    elif ranking >= 201 and ranking <=300:
        return '3rd Tier Top University'
    else:
        return 'Other Top University'

In [4]:
# Apply function
df['rank_level'] = df['world_rank'].apply(lambda x: create_category(x))
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,rank_level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,1st Tier Top University
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,1st Tier Top University
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,1st Tier Top University
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,1st Tier Top University
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,1st Tier Top University


A pivot table allows to pivot out one of these columns a new column headers and compare it against another column as row indices. Let's compare the new 'rank_level' (columns) versus the country of the universities (index) in terms of overall score (values). Then, we specify that the aggregation function is `np.mean` to get the average rating for universities in that country.

In [5]:
df.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean]).head()

mean                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
Argentina                      NaN                     NaN   
Australia                  47.9425                 49.2425   
Austria                        NaN                     NaN   
Belgium                    51.8750                 49.0840   
Brazil                         NaN                 49.5650   

                                                         
rank_level 3rd Tier Top University Other Top University  
country                                                  
Argentina                      NaN            44.672857  
Australia                47.285000            44.645750  
Austria                  47.066667            44.864286  
Belgium                  46.746667            45.081000  
Brazil                         NaN            44.499706

Pivot tables are not limited to one function that you might want to apply. You can pass in a list of different functions to apply: `aggfunc=[np.mean, np.max]`

In [6]:
df.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean, np.max]).head()

mean                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
Argentina                      NaN                     NaN   
Australia                  47.9425                 49.2425   
Austria                        NaN                     NaN   
Belgium                    51.8750                 49.0840   
Brazil                         NaN                 49.5650   

                                                         \
rank_level 3rd Tier Top University Other Top University   
country                                                   
Argentina                      NaN            44.672857   
Australia                47.285000            44.645750   
Austria                  47.066667            44.864286   
Belgium                  46.746667            45.081000   
Brazil                         NaN            44.499706   

                              amax                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
Argentina                      NaN                     NaN   
Australia                    51.61                   50.40   
Austria                        NaN                     NaN   
Belgium                      52.03                   49.73   
Brazil                         NaN                   49.82   

                                                         
rank_level 3rd Tier Top University Other Top University  
country                                                  
Argentina                      NaN                45.66  
Australia                    47.47                45.97  
Austria                      47.78                46.29  
Belgium                      47.14                46.21  
Brazil                         NaN                46.08

We can also summarise the values within a given top level column: `margins=True`

In [7]:
df.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean, np.max],
               margins=True).head()

mean                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
Argentina                      NaN                     NaN   
Australia                  47.9425                 49.2425   
Austria                        NaN                     NaN   
Belgium                    51.8750                 49.0840   
Brazil                         NaN                 49.5650   

                                                                    \
rank_level 3rd Tier Top University Other Top University        All   
country                                                              
Argentina                      NaN            44.672857  44.672857   
Australia                47.285000            44.645750  45.825517   
Austria                  47.066667            44.864286  45.139583   
Belgium                  46.746667            45.081000  47.011000   
Brazil                         NaN            44.499706  44.781111   

                              amax                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
Argentina                      NaN                     NaN   
Australia                    51.61                   50.40   
Austria                        NaN                     NaN   
Belgium                      52.03                   49.73   
Brazil                         NaN                   49.82   

                                                                
rank_level 3rd Tier Top University Other Top University    All  
country                                                         
Argentina                      NaN                45.66  45.66  
Australia                    47.47                45.97  51.61  
Austria                      47.78                46.29  47.78  
Belgium                      47.14                46.21  52.03  
Brazil                         NaN                46.08  49.82

A pivot table is just a multi-level DataFrame:

In [8]:
new_df = df.pivot_table(values='score', index='country', columns='rank_level', aggfunc=[np.mean, np.max],
               margins=True)

print(new_df.index)
print(new_df.columns) # Columns are hierarchical

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean', '1st Tier Top University'),
            ('mean', '2nd Tier Top University'),
            ('mean', '3rd Tier Top University'),
            ('mean',    'Other Top U

We can query the average scores of '1st Tier Top University' in each country by making the projections: first, for the mean, and then for the top tier category.

In [9]:
new_df['mean']['1st Tier Top University'].head() # Series

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: 1st Tier Top University, dtype: float64

In [11]:
# Or...
new_df[('mean', '1st Tier Top University')].head()

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: (mean, 1st Tier Top University), dtype: float64

To find the maximum average score on 1st Tier University level, we use: `idxmax()`

In [12]:
new_df[('mean', '1st Tier Top University')].idxmax()

'United Kingdom'

**Stack and unstack**:

To achieve a different shape of your pivot table, you can do it with the `stack()` and `unstack()` functions:

* stacking is pivoting the lowermost column index to become the innermost row index.

* unstacking is the inverse, pivoting the innermost row index to become the lowermost column index


In [13]:
new_df.head()

mean                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
Argentina                      NaN                     NaN   
Australia                  47.9425                 49.2425   
Austria                        NaN                     NaN   
Belgium                    51.8750                 49.0840   
Brazil                         NaN                 49.5650   

                                                                    \
rank_level 3rd Tier Top University Other Top University        All   
country                                                              
Argentina                      NaN            44.672857  44.672857   
Australia                47.285000            44.645750  45.825517   
Austria                  47.066667            44.864286  45.139583   
Belgium                  46.746667            45.081000  47.011000   
Brazil                         NaN            44.499706  44.781111   

                              amax                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
Argentina                      NaN                     NaN   
Australia                    51.61                   50.40   
Austria                        NaN                     NaN   
Belgium                      52.03                   49.73   
Brazil                         NaN                   49.82   

                                                                
rank_level 3rd Tier Top University Other Top University    All  
country                                                         
Argentina                      NaN                45.66  45.66  
Australia                    47.47                45.97  51.61  
Austria                      47.78                46.29  47.78  
Belgium                      47.14                46.21  52.03  
Brazil                         NaN                46.08  49.82

In [14]:
# stack
new_df = new_df.stack()
new_df.head() # Columns transpose into the rows

mean   amax
country   rank_level                               
Argentina Other Top University     44.672857  45.66
          All                      44.672857  45.66
Australia 1st Tier Top University  47.942500  51.61
          2nd Tier Top University  49.242500  50.40
          3rd Tier Top University  47.285000  47.47

In [16]:
# unstack
new_df.unstack().head() # Restores to the original

mean                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
All                      58.350675                49.06545   
Argentina                      NaN                     NaN   
Australia                47.942500                49.24250   
Austria                        NaN                     NaN   
Belgium                  51.875000                49.08400   

                                                                    \
rank_level 3rd Tier Top University Other Top University        All   
country                                                              
All                      46.843450            44.738871  47.798395   
Argentina                      NaN            44.672857  44.672857   
Australia                47.285000            44.645750  45.825517   
Austria                  47.066667            44.864286  45.139583   
Belgium                  46.746667            45.081000  47.011000   

                              amax                          \
rank_level 1st Tier Top University 2nd Tier Top University   
country                                                      
All                         100.00                   51.29   
Argentina                      NaN                     NaN   
Australia                    51.61                   50.40   
Austria                        NaN                     NaN   
Belgium                      52.03                   49.73   

                                                                 
rank_level 3rd Tier Top University Other Top University     All  
country                                                          
All                          47.93                46.34  100.00  
Argentina                      NaN                45.66   45.66  
Australia                    47.47                45.97   51.61  
Austria                      47.78                46.29   47.78  
Belgium                      47.14                46.21   52.03

In [17]:
# what if we double unstack()
new_df.unstack().unstack().head() # We end up with a single column (a Series)

      rank_level               country  
mean  1st Tier Top University  All          58.350675
                               Argentina          NaN
                               Australia    47.942500
                               Austria            NaN
                               Belgium      51.875000
dtype: float64